In [ ]:
from torchvision import datasets, transforms

DATASET_DIR = "../Rock-Paper-Scissors/train"

transform = transforms.Compose([
    transforms.Resize((64, 64)),   
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(DATASET_DIR, transform=transform)

print("Classi:", dataset.classes)
print("Numero immagini:", len(dataset))


Classi: ['paper', 'rock', 'scissors']
Numero immagini: 2520


CONTROLLO LE DIMENSIONI DELLE IMMAGINI 

In [3]:
img, label = dataset[0]
print("Dimensione immagine:", img.shape)


Dimensione immagine: torch.Size([3, 64, 64])


CREO IL DATALOADER 

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

class RPSNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.net(x)

model = RPSNet().to(DEVICE)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss:.3f}")


Epoch 1/10 - Loss: 50.053
Epoch 2/10 - Loss: 3.776
Epoch 3/10 - Loss: 0.825
Epoch 4/10 - Loss: 0.376
Epoch 5/10 - Loss: 0.129
Epoch 6/10 - Loss: 0.082
Epoch 7/10 - Loss: 0.051
Epoch 8/10 - Loss: 0.035
Epoch 9/10 - Loss: 0.025
Epoch 10/10 - Loss: 0.020


In [ ]:
torch.save(model.state_dict(), "rps_model.pth")
print("Modello salvato!")


✅ Modello salvato!
